In [41]:
# !pip install -U mlflow

In [43]:
from xgboost import XGBClassifier
import pandas as pd
import torch
import mlflow
from mlflow import log_artifacts, log_params, log_metrics

In [6]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
# sampled_training = pd.read_csv('./drive/MyDrive/VK_hack/final_bert_sampled_train_df.csv')


sampled_training = pd.read_csv('./drive/MyDrive/VK_hack/df_extra_gen_old.csv')
tr_indx = np.load('./drive/MyDrive/VK_hack/tr_indx.npy')
vl_indx = np.load('./drive/MyDrive/VK_hack/vl_indx.npy')

In [66]:
train_df = sampled_training.copy()

In [67]:
label_map = {
    'Красота': 0,
    'Политика': 1,
    'Экономика': 2,
    'Мода': 3,
    'Общество': 4,
    'Закон и право': 5,
    'Кино': 6,
    'Телевидение': 7,
    'Персоны': 8,
    'События': 9,
    'Бренды': 10,
    'Наука': 11,
    'Гаджеты': 12,
    'Соцсети': 13,
    'Дом': 14,
    'Технологии': 15,
    'Транспорт': 16,
    'Погода': 17,
    'Рецепты': 18,
    'Опросы': 19,
    'Головоломки': 20
}

In [68]:
labels_t = train_df.loc[:, label_map.keys()].values

In [69]:
labels_t.shape

(97241, 21)

### Embedder

In [9]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [10]:
#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
# #Sentences we want sentence embeddings for
# sentences = ['Привет! Как твои дела?',
#              'А правда, что 42 твое любимое число?']

In [11]:
texts_list = list(train_df.loc[:, 'X'].str.replace(u'\xa0', u' ').values)

In [12]:
import numpy as np
from tqdm import tqdm

results = np.zeros(shape=(len(texts_list), 1024))

In [13]:
from torch.utils.data import DataLoader, TensorDataset

device = 'cuda'
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(120138, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inp

In [14]:
texts_list[0]

'Ирина Шейк разделась для рекламы джинсов Российская супермодель Ирина Шейк приняла участие в рекламной кампании французского модного дома Givenchy и представила линейку джинсовой одежды. На снимках россиянка предстала почти обнаженной. Первые кадры с фотосесссии в своем Instagram опубликовал креативный директор компании Риккардо Тиши. Шейк представила линейку одежды из денима, о запуске которой было объявлено в сентябре 2015 года. Ее партнером по съемочной площадке стал манекенщик Крис Мур. Бывшая спутница футболиста Криштиану Роналду с 2007 года снимается для журнала Sports Illustrated, также она сотрудничала с такими брендами, как Armani Exchange, Guess, Intimissimi, Beach Bunny, Lacoste и La Perla. Givenchy — французский модный дом, основан в 1952 году Юбером де Живанши. Специализируется на выпуске одежды, обуви, аксессуаров и парфюмерии. Клиентками дома были Одри Хэпберн и Жаклин Кеннеди. С 2005 года дизайнером дома является итальянский модельер Риккардо Тиши. В середине декабря в

In [15]:
for i in texts_list:
    if not isinstance(i, str):
        print(i)

In [16]:
# texts_list_first = texts_list[:len(texts_list) // 2]
# texts_list_second = texts_list[len(texts_list) // 2: ]

In [ ]:
tokenized_texts = tokenizer(texts_list, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Create DataLoader
dataset = TensorDataset(tokenized_texts["input_ids"], tokenized_texts["attention_mask"])
dataloader = DataLoader(dataset, batch_size=int(128))

In [17]:
from tqdm import tqdm
import torch

In [18]:
# # Generate embeddings
# embeddings = []
# with torch.no_grad():
#     for input_ids, attention_mask in tqdm(dataloader):
#         input_ids = input_ids.to(device)
#         attention_mask = attention_mask.to(device)
#         outputs = model(input_ids, attention_mask=attention_mask)
#         sentence_embeddings = mean_pooling(outputs, attention_mask)
#         embeddings.append(sentence_embeddings.cpu())

# # Concatenate embeddings
# first_embeddings = torch.cat(embeddings)

In [ ]:
# torch.save(first_embeddings, './drive/MyDrive/VK_hack/sentence_embeddings.pt')

### XGBoost

In [70]:
sampled_training.columns

Index(['url', 'title', 'text', 'topic', 'tags', 'date', 'Красота', 'Политика',
       'Экономика', 'Мода', 'Общество', 'Закон и право', 'Кино', 'Телевидение',
       'Персоны', 'События', 'Бренды', 'Наука', 'Гаджеты', 'Соцсети', 'Дом',
       'Технологии', 'Транспорт', 'X', 'Погода', 'Рецепты', 'Опросы',
       'Головоломки', 'labels', 'actual_labels', 'clean_X', 'cut_labels'],
      dtype='object')

In [71]:
first_embeddings = torch.load('./drive/MyDrive/VK_hack/sentence_embeddings.pt')
extra_embeddings = torch.load('./drive/MyDrive/VK_hack/extra_embeddings.pt')

In [73]:
embeddings = np.concatenate([first_embeddings.numpy(), extra_embeddings.numpy()], axis=0)

In [74]:
embeddings.shape

(97241, 1024)

In [77]:
# use previous model split

# Split the data into train and test sets
X_train, X_test = embeddings[tr_indx], embeddings[vl_indx]
y_train, y_test = labels_t[tr_indx], labels_t[vl_indx]

In [78]:
# Initialize XGBoost classifier
model = XGBClassifier()

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.675767391639673


In [88]:
import pickle
file_name = "./drive/MyDrive/VK_hack/xgb_clf.pkl"

# save
pickle.dump(model, open(file_name, "wb"))

# load
# xgb_model_loaded = pickle.load(open(file_name, "rb"))

In [90]:
sampled_training.iloc[vl_indx].iloc[0].X

'Путин высказался о\xa0свободном интернете Интернет должен оставаться свободным, а для противодействия появляющимся угрозам важен диалог с сообществом сети, считает президент России Владимир Путин. Его слова приводятся в приветствии к участникам и гостям X Российского форума по управлению интернетом (RIGF-2019), передает ТАСС в понедельник, 8 апреля. Послание Путина зачитал первый заместитель руководителя администрации президента России Сергей Кириенко. «Убежден, мы должны и впредь следовать принципу свободы интернета, обеспечить все условия для широкого обмена информацией, для реализации бизнес-инициатив и стартапов», — считает российский лидер. По его мнению, для противодействия распространению контента, представляющего угрозу правам граждан и интересам государства, важное значение имеет позиция интернет-сообщества, «его открытость и готовность к конструктивному диалогу». Президент напомнил, что 7 апреля 1994 года был официально зарегистрирован национальный домен .ru для РФ. По его с

In [116]:
sampled_training[sampled_training.X.str.len() == 100].X.values[0]

'Закрытие пикетов в России и их последствия для свободы словаНаписал ли ты что-то новое и интересное?'

In [91]:
y_tst_pred[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0.])

In [79]:
from sklearn.metrics import f1_score

In [80]:
y_tst_pred = model.predict(X_test)
f1_micro = f1_score(y_test, y_tst_pred, average='micro')

In [107]:
np.save('./drive/MyDrive/VK_hack/xgb_preds.npy', y_tst_pred)

In [100]:
mask = [True if i == 1 else False for i in y_tst_pred[0]]

In [108]:
mask

[False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [105]:
np.array(list(label_map.keys()))[mask]

array(['Политика', 'Соцсети'], dtype='<U13')

In [81]:
f1_marco = f1_score(y_test, y_tst_pred, average='macro')

In [82]:
metrics = dict(f1_micro=f1_micro,
               f1_marco=f1_marco)

In [83]:
metrics

{'f1_micro': 0.8103991237078113, 'f1_marco': 0.8391256898789453}

In [38]:
y_test.sum(axis=1)

array([1., 2., 2., ..., 2., 1., 1.])

In [51]:
import os
os.environ['MLFLOW_TRACKING_USERNAME']='admin'
os.environ['MLFLOW_TRACKING_PASSWORD']='4c7nD5JhP49B62a|'

In [84]:
# mlflow.set_tracking_uri("https://mlp-9915-mlp-3ce14c984edd90.ml.msk.vkcs.cloud/mlflow:5000")
# experiment = mlflow.set_experiment('tag_multilabel')
# with mlflow.start_run(experiment_id=experiment.experiment_id, run_name='xgboost'):
#     log_metrics(metrics)